# Exposure age calculation

## The concentration at the surface

Cosmogenic radionuclides accumulate at the Earth surface and can be modelled as a function of exposure duration and erosion rates. In this notebook, we will explore these interactions. 

First, run the cell below to set the working environment (no need to understand these lines of code). If Google Colab warns about the trustfullness of the code, click on `Run anyway`.

In [1]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import display
from IPython import get_ipython
import os, sys, platform, shutil

# Define root directory according to current environment
if 'google.colab' in str(get_ipython()):

    dir_root = "/content/cosmo-training"

    if os.path.exists(dir_root):
      shutil.rmtree(dir_root)

    !git clone https://github.com/franz825/cosmo-training

else:
    # Path to root directory (repository)
    dir_root = os.path.dirname(sys.path[0])

# Import custom functions for CRN computation
sys.path.append(dir_root)
from src.functions.get_parameters_values import get_parameters_values
from src.functions.compute_surface_concentration import compute_surface_concentration
from src.functions.compute_exposure_age import compute_exposure_age

The parameters used as inputs to model cosmogenic nuclides are defined by the function `parameters_values()`.

If needed, you can modify these values directly in the function `.py` file and re-run the notebook from the beginning.

In [ ]:
# Get parameters values defined in the function.
p = get_parameters_values()
display(p)

To compute the <sup>10</sup>Be surface concentration, the equation below is used (Dunai, 2010). We asssume that exposure time is clocking since the onset of the erosion of the surface, with no pre-exposure memory.

$$ C = \frac{P}{\lambda + \frac{\epsilon \cdot \rho}{\Lambda}} \cdot \Bigg[1 - \exp\Bigg(-T_{exp} \cdot \Big(\lambda + \frac{\epsilon \cdot \rho}{\Lambda}\Big)\Bigg)\Bigg] $$

With:
- $\lambda$: Decay constant (1/yr)
- $P$: Production rate (at/g/yr)
- $\rho$: Bulk density (g/cm³)
- $\Lambda$: Attenuation length (g/cm²)
- $T_{exp}$: Exposure time (yr)
- $\epsilon:$ Erosion rate (cm/yr)

Let's compute the <sup>10</sup>Be concentration of a surface eroding at 0.01 cm/yr (or 1 m/Ma) for 10,000 yr: 

In [ ]:
# 10Be concentration at the surface (at/g)
erosion_rate = 0.01
exposure_age = 100000

concentration = compute_surface_concentration(erosion_rate, exposure_age, p)

# Return the result
print(concentration)

Re-run the previous Python cell with other values of erosion rates and exposure ages to evaluate their impact on modelled <sup>10</sup>Be concentrations.

Let's now build a graph showing the relation between exposure age and <sup>10</sup>Be surface concentration for different values of erosion rates. 
Here below, you can define values of interest for erosion rate (cm/yr) and exposure age. 

In [ ]:
# Constant erosion rate of the surface (cm/yr)
erosion_rates = [0.000001, 0.0001, 0.001, 0.03]

# Vector containing the exposure ages (yr) to process
exposure_ages = [1000, 2000, 5000, 10000, 20000, 30000, 50000, 100000, 200000, 300000, 400000, 500000, 1000000]

# Print values
print("Erosion rates (cm/yr): ")
print(erosion_rates)
print("Exposure ages (a): ")
print(exposure_ages)

Run the code below to compute the <sup>10</sup>Be concentration using a loop over the erosion rate and exposure age values defined above. 

In [ ]:
# Containers for 10Be concentrations results
crn_table = pd.DataFrame({"exposure_age": exposure_ages})
crn_plot = pd.DataFrame()

# Loop within erosion rate values
for i in range(len(erosion_rates)):

    # Get current erosion rate
    erosion_rate = erosion_rates[i]

    # Array to contain CRN concentration for each 
    crn_concentrations = np.zeros(len(exposure_ages))

    # Loop within exposure age values
    for i in range(len(exposure_ages)): 
        
        # Get current exposure age
        exposure_age = exposure_ages[i]

        # Compute 10Be concentration at the surface 
        Be_concentration = compute_surface_concentration(erosion_rate, exposure_age, p)

        # Divide 10Be concentration by 1000 (and store in array
        crn_concentrations[i] = Be_concentration

    # Convert array of concentrations in a named serie
    crn_concentrations = pd.Series(crn_concentrations, name = "crn_conc_erate_" + str(erosion_rate), dtype = 'Float64')
    # Include array of concentrations in a complete dataframe
    crn_df = pd.DataFrame({"exposure_age": exposure_ages, "crn_concentration": crn_concentrations, "erosion_rate": pd.Series(np.repeat(erosion_rate, len(exposure_ages)))})

    # Collect concentrations formatted for plots
    crn_plot = pd.concat([crn_plot, crn_df], axis = 0)
    # Collect concentrations into pandas dataframe for csv export
    crn_table = pd.concat([crn_table, crn_concentrations], axis = 1)

Results are stored in two objects: `crn_table` that can be exported to .csv file for external use, and `crn_plot` that will be used below to plot the final graph.

In [ ]:
# Write results to csv file
crn_table.to_csv("exposure_ages.csv")

# Print the table containing 10Be concentration for each exposure age
display(crn_table)

In [ ]:
# Convert erosion rate to m/Ma and coerce as string. 
crn_plot["erosion_rate"] = crn_plot["erosion_rate"] * 10000
crn_plot["erosion_rate"] = crn_plot["erosion_rate"].astype(str)
# Convert concentrations from at/g to 10^3 at/g.
crn_plot["crn_concentration"] = crn_plot["crn_concentration"] / 1000

# Plot 10Be concentrations for various erosion rates 
plot = px.scatter(crn_plot, x = "exposure_age", y = "crn_concentration", color = "erosion_rate", labels = dict(exposure_age = "Exposure age (a)", 
crn_concentration = "Concentration (10<sup>3</sup> at/g)", erosion_rate = "Erosion rate (m /Ma)"), log_x = True, log_y = True)
plot.show()

## The exposure dating

We sample a shallow granite block (density = 2.7) and measure a <sup>10</sup>Be concentration of 100 x 10<sup>3</sup> at/g. Let's consider a production rate at sea level high latitude (SLHL) of 1 at/g/yr and the <sup>10</sup>Be half live of 1.37 x 10<sup>6</sup> yr. 

Exposure age can be written as follows: 

$$ T_{exp} = - \frac{1}{\lambda + \frac{\rho \cdot \epsilon}{\Lambda}} \cdot ln\Bigg[ 1-\frac{C \cdot \big(\lambda + \frac{\rho \cdot \epsilon}{\Lambda}\big)}{P} \Bigg] $$

With:
- $\lambda$: Decay constant (1/yr)
- $P$: Production rate (at/g/yr)
- $\rho$: Bulk density (g/cm³)
- $\Lambda$: Attenuation length (g/cm²)
- $\epsilon:$ Erosion rate (cm/yr)
- $C$: Cosmogenic concentration (at/g)

Using the function `compute_exposure_duration`, compute the exposure age for an erosion rate of 1, 2, 5, 10, and 50 m/Myr. 

How is the exposure age evolving in function of the erosion rate ? Why ?

In [ ]:
# 10Be concentration measured in the sample (at/g)
surface_concentration = 1000
# Erosion rate (cm/yr)
erosion_rate = 0.0001

# Compute exposure age (yr)
exposure_age = compute_exposure_age(surface_concentration, erosion_rate, p)

print(exposure_age)